# User for Genre

- User for genre toma como parámetro un género particular, y retorna el usuario  con mayor tiempo jugado para dicho genero.

- Para evitar consultas lentas, crearemos una tabla auxiliar que sera agrupada por año y jugador, donde agruparemos los generos en un str y sumaremos el tiempo jugado del usuario.

In [1]:
import pandas as pd
import os

path_user = os.path.join('..','data','clear','users_items.csv.gz')
path_steam_games = os.path.join('..','data','clear','steam_games.csv.gz')

## Carga de data

 """"   el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
"""

In [2]:
path_user = os.path.join('..','data','clear','users_items.csv.gz')
path_steam_games = os.path.join('..','data','clear','steam_games.csv.gz')

## Cargamos la data necesaria para solucionar este problema.
steam_colums = ['id','genres','release_year']
steam_games = pd.read_csv(path_steam_games, usecols = steam_colums)

## renombraremos la columna id por steam_id
steam_games.rename(columns={'id':'steam_id'}, inplace=True)

user_item_cols = ['item_id','user_id','playtime_forever']
user_items = pd.read_csv(path_user, usecols = user_item_cols)


In [3]:
steam_games.head(3)

,genres,steam_id,release_year
0,"Strategy, Action, Indie, Casual, Simulation",761140.0,2018
1,"Free to Play, Strategy, Indie, RPG, Card Game,...",643980.0,2018
2,"Free to Play, Simulation, Sports, Casual, Indi...",670290.0,2017


In [4]:
user_items.head(3)

,user_id,playtime_forever,item_id
0,76561197970982479,6.0,10.0
1,76561197970982479,0.0,20.0
2,76561197970982479,7.0,30.0


## Creación de tabla consulta

- En esta sección combinaremos steam_games  con user_items, por medio de sus ids.
Luego agruparemos por año de y grupo de genero comun, para calcular la suma de tiempos jugados por dicho genero.

In [5]:

steam_users = steam_games.merge(user_items,
                              left_on='steam_id', 
                              right_on= 'item_id',
                              how='inner').drop(columns=['steam_id','item_id'])

In [6]:
steam_users.sample(5)

,genres,release_year,user_id,playtime_forever
2305042,"Early Access, Free to Play, Sandbox, Multiplay...",2014,Lorazx,6.0
1298808,"Early Access, Free to Play, Shooter, Multiplay...",2014,76561198143395684,2584.0
2060686,"RPG, Adventure, Action, Indie, Voxel, Co-op, D...",2015,SubmarineBear,346.0
433774,"Action, Adventure, Hack and Slash, Third Perso...",2010,76561198043305219,0.0
2096077,"World War I, FPS, Multiplayer, War, Historical...",2015,76561198052408414,1375.0


- Por eficiencia en la consulta vamos a agrupar por año de lanzamiento y usuario.
Notamos que quedan dos columans sin agrupar, genres (str) y playtime_forever. Al realizar la agrupación por suma, vamos a acumular en playtime_forever las horas jugadas por un usuario en un año determinado; por otra parte genres concatenará cada string que contiene las columnas agrupadas. Esta técnica nos permitira seguir teniendo toda la información necesaria. Posteriormente vamos a transformar la columna genres para eliminar los generos repetidos en cada registro.

In [7]:
tabla_endpoint_2 = steam_users.groupby(['release_year','user_id']).sum().reset_index()

In [8]:
tabla_endpoint_2

,release_year,user_id,genres,playtime_forever
0,1983,2Ta4,"Adventure, Action, FMV, Casual, Quick-Time Eve...",18.0
1,1983,76561197960324641,"Adventure, Action, FMV, Casual, Quick-Time Eve...",0.0
2,1983,76561197966936422,"Adventure, Action, FMV, Casual, Quick-Time Eve...",331.0
3,1983,76561197968887720,"Adventure, Action, FMV, Casual, Quick-Time Eve...",1.0
4,1983,76561197969020980,"Adventure, Action, FMV, Casual, Quick-Time Eve...",98.0
...,...,...,...,...
763521,2018,xerector,"Early Access, Indie, Platformer, Metroidvania",0.0
763522,2018,xinyo,"Early Access, Action, Indie, SimulationEarly A...",0.0
763523,2018,xmatiias,"Early Access, Action, Adventure, RPG, Indie, A...",0.0
763524,2018,yes12429,"Early Access, Indie, Platformer, Metroidvania",0.0


- Así observamos que la tabla resultante tendra un tamaño considerablemente menor y cada usuario que se encuentre en estas lista, tendra los generos jugados por este usuario.

- Ahora vamos a eliminar los generos repetidos en cada registro, esto con el fin de reducir el tamaño del string de cada registro de genres. dado que esta consulta consume bastante memoria, utilicé una tecnica de carga con generadores.

In [9]:
## tomar los generos diferentes de cada uno de los caracteres
def genres_unique(x):
  return ', '.join(set(map(str.strip, x.split(', '))))

tabla_endpoint_2['genres'] = list(genres_unique(fila) for fila in tabla_endpoint_2['genres'])

In [10]:
tabla_endpoint_2.head(10)

,release_year,user_id,genres,playtime_forever
0,1983,2Ta4,"FMV, Short, Action, Retro, Cartoon, Classic, A...",18.0
1,1983,76561197960324641,"FMV, Short, Action, Retro, Cartoon, Classic, A...",0.0
2,1983,76561197966936422,"FMV, Short, Action, Retro, Cartoon, Classic, A...",331.0
3,1983,76561197968887720,"FMV, Short, Action, Retro, Cartoon, Classic, A...",1.0
4,1983,76561197969020980,"FMV, Short, Action, Retro, Cartoon, Classic, A...",98.0
5,1983,76561197971401137,"FMV, Short, Action, Retro, Cartoon, Classic, A...",33.0
6,1983,76561197971951483,"FMV, Short, Action, Retro, Cartoon, Classic, A...",27.0
7,1983,76561197972452208,"FMV, Short, Action, Retro, Cartoon, Classic, A...",0.0
8,1983,76561197973470219,"FMV, Short, Action, Retro, Cartoon, Classic, A...",0.0
9,1983,76561197975369524,"FMV, Short, Action, Retro, Cartoon, Classic, A...",45.0


## Exportar tabla consulta 2

In [12]:
path_endpoint_2 = os.path.join('..','data','clear','02_user_for_genre_data.csv.gz')


tabla_endpoint_2.to_csv(path_endpoint_2, index = False,compression='gzip')

## Lectura de tabla consulta

**Retornaremos el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año**

In [15]:
table_2 = pd.read_csv(path_endpoint_2)

## Construcción de la consulta
- A continuación vamos a realizar las consultas necesarias para encontrar el usuario con mas horas jugadas por genero.
Para responder este problema vamos inspecciónar con un genero particular, en este caso **Action**, luego vamos a contruir las consultas para este genero particular

In [16]:
genre = 'Action'

In [17]:
# Filtro la tabla para quedarme únicamente con registros que contengan el género deseado
genre_filter = table_2[table_2['genres'].str.contains(genre)]


In [18]:
genre_filter.head(4)

,release_year,user_id,genres,playtime_forever
0,1983,2Ta4,"FMV, Short, Action, Retro, Cartoon, Classic, A...",18.0
1,1983,76561197960324641,"FMV, Short, Action, Retro, Cartoon, Classic, A...",0.0
2,1983,76561197966936422,"FMV, Short, Action, Retro, Cartoon, Classic, A...",331.0
3,1983,76561197968887720,"FMV, Short, Action, Retro, Cartoon, Classic, A...",1.0


In [23]:
## Sumamos playtime_forever de cada usuario 
df_sum_play_time = genre_filter.groupby('user_id')['playtime_forever'].sum()

In [27]:
pd.DataFrame(df_sum_play_time)

,playtime_forever
user_id,
--000--,140864.0
--ace--,70359.0
--ionex--,38584.0
-2SV-vuLB-Kg,48221.0
-404PageNotFound-,198665.0
...,...
zzonci,429.0
zzoptimuszz,78498.0
zzydrax,4640.0


In [34]:
# Usuario con más horas jugadas en total
user_max_time = df_sum_play_time.idxmax()

print(f'El usurio con mas horas jugadas \npara el genero {genre} es {user_max_time}')


El usurio con mas horas jugadas 
para el genero Action es REBAS_AS_F-T


In [35]:
# Filtrar solo por los registros del usuario con mayor tiempo de juego y me quedo con las columnas release_year y playtime_forever
mask = genre_filter['user_id'] == user_max_time

user_filter = genre_filter[mask][['release_year', 'playtime_forever']]

In [36]:
user_filter.head(5)

,release_year,playtime_forever
17153,1996,0.0
36546,1998,1.0
50057,1999,225.0
64835,2000,0.0
79718,2001,11.0


In [37]:
# Sumar las horas jugadas del usuario por año
user_filter_sum = user_filter.groupby('release_year')['playtime_forever'].sum().cumsum()

In [41]:
# Este es el tiempo acumulado jugado por el usuario con mas tiempo jugado con el genero action.
pd.DataFrame(user_filter_sum)

,playtime_forever
release_year,
1996,0.0
1998,1.0
1999,226.0
2000,226.0
2001,237.0
2002,238.0
2003,2101.0
2004,11203.0
2005,14496.0


In [42]:
# Retornaremos como una lista de diccionarios del estilo
# {year : playtime_forever_cum }

target = user_filter_sum.reset_index().rename(columns={'release_year': 'year', 'playtime_forever': 'total_horas'}).to_dict(orient='records')

In [49]:
target[:5]

[{'year': 1996, 'total_horas': 0.0},
 {'year': 1998, 'total_horas': 1.0},
 {'year': 1999, 'total_horas': 226.0},
 {'year': 2000, 'total_horas': 226.0},
 {'year': 2001, 'total_horas': 237.0}]

In [58]:
print(f"Usuario con más horas jugadas para Género {genre}: es {user_max_time}, \nHoras jugadas:")
target

Usuario con más horas jugadas para Género Action: es REBAS_AS_F-T, 
Horas jugadas:


[{'year': 1996, 'total_horas': 0.0},
 {'year': 1998, 'total_horas': 1.0},
 {'year': 1999, 'total_horas': 226.0},
 {'year': 2000, 'total_horas': 226.0},
 {'year': 2001, 'total_horas': 237.0},
 {'year': 2002, 'total_horas': 238.0},
 {'year': 2003, 'total_horas': 2101.0},
 {'year': 2004, 'total_horas': 11203.0},
 {'year': 2005, 'total_horas': 14496.0},
 {'year': 2006, 'total_horas': 16343.0},
 {'year': 2007, 'total_horas': 26769.0},
 {'year': 2008, 'total_horas': 39671.0},
 {'year': 2009, 'total_horas': 101393.0},
 {'year': 2010, 'total_horas': 174351.0},
 {'year': 2011, 'total_horas': 360568.0},
 {'year': 2012, 'total_horas': 628128.0},
 {'year': 2013, 'total_horas': 1036426.0},
 {'year': 2014, 'total_horas': 1516004.0},
 {'year': 2015, 'total_horas': 2589860.0},
 {'year': 2016, 'total_horas': 3912840.0},
 {'year': 2017, 'total_horas': 3947907.0}]

## Función UserForGenre

In [23]:
def user_for_genre(genre :str):
  
  table_2 = pd.read_csv('entpoint_2.csv')

  ## Filtro la tabla para quedarme unicamente con registros que contengan el genero deseado
  genre_filter = table_2[table_2['genres'].str.contains(genre)]
  
  ## usuario con mayor tiempo de juego acumulado
  df_sum_play_time =  genre_filter[['user_id','playtime_forever']].groupby(['user_id']).sum()
  
  # usuario con mas horas jugadas en total
  if len(df_sum_play_time) > 0:
    user_max_time = df_sum_play_time.idxmax().iloc[0]
  else:
    return ('No existe usuario que haya jugado este juego')
  
  ## Filtraremos solo por los registros del usuario con mayor tiempo de juego
  ## nos quedamos con el año de lanzamiento y el tiempo de juego
  mask = genre_filter['user_id'] == user_max_time
  user_filter = genre_filter[mask][['release_year','playtime_forever']]
  
  # Sumamos las horas jugadas del usuario por año
  user_filter_sum = user_filter.groupby('release_year').sum().reset_index()
  
  # ## Calculamos la suma acumulada
  user_filter_sum['total_horas'] = user_filter_sum['playtime_forever'].cumsum()
  
  target = user_filter_sum.rename(columns={'release_year':'year'})[['year', 'total_horas']].to_dict(orient='records')
  return {f"Usuario con más horas jugadas para Género {genre}" : user_max_time, "Horas jugadas": target}

In [24]:
user_for_genre('Action')


{'Usuario con más horas jugadas para Género Action': 'REBAS_AS_F-T',
 'Horas jugadas': [{'year': 1996, 'total_horas': 0.0},
  {'year': 1998, 'total_horas': 1.0},
  {'year': 1999, 'total_horas': 226.0},
  {'year': 2000, 'total_horas': 226.0},
  {'year': 2001, 'total_horas': 237.0},
  {'year': 2002, 'total_horas': 238.0},
  {'year': 2003, 'total_horas': 2101.0},
  {'year': 2004, 'total_horas': 11203.0},
  {'year': 2005, 'total_horas': 14496.0},
  {'year': 2006, 'total_horas': 16343.0},
  {'year': 2007, 'total_horas': 26769.0},
  {'year': 2008, 'total_horas': 39671.0},
  {'year': 2009, 'total_horas': 101393.0},
  {'year': 2010, 'total_horas': 174351.0},
  {'year': 2011, 'total_horas': 360568.0},
  {'year': 2012, 'total_horas': 628128.0},
  {'year': 2013, 'total_horas': 1036426.0},
  {'year': 2014, 'total_horas': 1516004.0},
  {'year': 2015, 'total_horas': 2589860.0},
  {'year': 2016, 'total_horas': 3912840.0},
  {'year': 2017, 'total_horas': 3947907.0}]}